[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/examples/pinecone-and-scaling.ipynb)

# Scaling to Many Routes and Using Pinecone

Semantic router can be used with many hundreds, thousands, or even more routes. At very large scales it can be useful to use a vector database to store and search though your route vector space. Although we do not demonstrate _very large_ scale in this notebook, we will demonstrate more routes than usual and we will also see how to use the `PineconeIndex` for potential scalability and route persistence beyond our local machines.

## Installing the Library

In [1]:
!pip install -qU \
    "semantic-router[local, pinecone]==0.0.22" \
    datasets==2.17.0


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Downloading Routes

In [2]:
from datasets import load_dataset

data = load_dataset("aurelio-ai/generic-routes", split="train")
data

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['name', 'utterances', 'description', 'function_schema', 'llm', 'score_threshold'],
    num_rows: 50
})

Each row in this dataset is a single route:

In [3]:
data[0]

{'name': 'politics',
 'utterances': ["isn't politics the best thing ever",
  "why don't you tell me about your political opinions",
  "don't you just love the presidentdon't you just hate the president",
  "they're going to destroy this country!",
  'they will save the country!'],
 'description': None,
 'function_schema': None,
 'llm': None,
 'score_threshold': 0.82}

We transform these into `Route` objects like so:

In [4]:
from semantic_router import Route

routes = [Route(**data[i]) for i in range(len(data))]
routes[0]

Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schema=None, llm=None, score_threshold=0.82)

Next we initialize an `encoder`. We will use a simple `HuggingFaceEncoder`, we can also use popular encoder APIs like `CohereEncoder` and `OpenAIEncoder`.

In [5]:
from semantic_router.encoders import HuggingFaceEncoder

encoder = HuggingFaceEncoder()

c:\Users\Siraj\Documents\Personal\Work\Aurelio\Virtual Environments\semantic_router_3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Now we initialize our `PineconeIndex`, all it requires is a [Pinecone API key](https://app.pinecone.io) (you do need to be using Pinecone Serverless).

In [6]:
import os
from getpass import getpass
from semantic_router.index.pinecone import PineconeIndex

os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

index = PineconeIndex(index_name="index", namespace="namespace")

2024-05-07 21:33:48 WARNING semantic_router.utils.logger Index could not be initialized.


In [7]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-05-07 21:33:48 INFO semantic_router.utils.logger local


We run the solely static routes layer:

In [8]:
rl("how's the weather today?").name

2024-05-07 21:33:57 WARNING semantic_router.utils.logger No classification found for semantic classifier.
2024-05-07 21:33:57 ERROR semantic_router.utils.logger No route found with name . Check to see if any Routes have been defined.


_If you see a warning about no classification being found, wait a moment and run the above cell again._

## Loading Index From Previous Initialization

Because we're using Pinecone our route index can now persist / be access from different locations by simply connecting to the pre-existing index, by default this index uses the identifier `"semantic-router--index"` — this is the index we'll be loading here, but we can change the name via the `index_name` parameter if preferred.

First, let's delete our old route layer, `index`, and `routes`.

In [9]:
del rl, index, routes

Let's load our index first. As mentioned, `"index"` is the default index name and we are passing `"namespace"` as namespace name for the pinecone object.

In [10]:
index = PineconeIndex(index_name="index", namespace="namespace")

We load the pre-existing routes from this index like so:

In [11]:
index.get_routes()

[('fitness_tips', 'suggest a workout routine'),
 ('career_advice', 'what are the emerging career fields?'),
 ('mental_health_support', 'how can I manage stress?'),
 ('gardening_and_horticulture', 'suggest some easy-care indoor plants'),
 ('fitness_tips', 'give me a fitness tip'),
 ('debugging_tips', 'tips for debugging asynchronous code'),
 ('historical_events', 'share an interesting piece of medieval history'),
 ('chitchat', 'the weather is horrendous'),
 ('art_and_culture', 'suggest some must-visit museums'),
 ('daily_inspiration', 'I need some inspiration for today'),
 ('language_learning', 'suggest ways to learn a new language'),
 ('creative_writing_and_literature', 'how can I improve my writing skills?'),
 ('cloud_computing', 'AWS vs Azure vs Google Cloud'),
 ('language_learning', 'how can I improve my Spanish?'),
 ('career_advice_in_tech', 'navigating career growth in tech'),
 ('creative_writing_and_literature', 'what are some tips for storytelling?'),
 ('data_structures_and_algo

We will transform these into a dictionary format that we can use to initialize our `Route` objects.

In [12]:
routes_dict = {}
for route, utterance in index.get_routes():
    if route not in routes_dict:
        routes_dict[route] = []
    routes_dict[route].append(utterance)

In [13]:
routes_dict

{'book_recommendations': ['suggest a good book to read',
  'I need a book recommendation',
  "what's your favorite book?"],
 'chitchat': ['lovely weather today',
  "how's the weather today?",
  'the weather is horrendous',
  "let's go to the chippy",
  'how are things going?'],
 'jokes': ['make me laugh', 'tell me a joke', 'know any good jokes?'],
 'frameworks_and_libraries': ["what's the difference between React and Angular?",
  'introduction to Django for web development',
  'best Python libraries for data analysis'],
 'language_learning': ['what are some effective language learning techniques?',
  'suggest ways to learn a new language',
  'how can I improve my Spanish?'],
 'fitness_tips': ['how can I stay active at home?',
  'suggest a workout routine',
  'give me a fitness tip'],
 'debugging_tips': ['tips for debugging asynchronous code',
  'how do I debug segmentation faults in C++?',
  'best tools for JavaScript debugging'],
 'interview_preparation': ['how to prepare for a coding

Now we transform these into a list of `Route` objects.

In [14]:
routes = [
    Route(name=route, utterances=utterances)
    for route, utterances in routes_dict.items()
]
routes[0]

Route(name='book_recommendations', utterances=['suggest a good book to read', 'I need a book recommendation', "what's your favorite book?"], description=None, function_schema=None, llm=None, score_threshold=None)

Now we reinitialize our `RouteLayer`:

In [15]:
from semantic_router import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=index)

2024-05-07 21:34:08 INFO semantic_router.utils.logger local


And test it again:

In [16]:
rl("say something to make me laugh").name

'jokes'

In [17]:
rl("tell me something amusing").name

'jokes'

In [18]:
rl("it's raining cats and dogs today").name

'chitchat'

In [19]:
# delete index
index.delete_index()

Perfect, our routes loaded from our `PineconeIndex` are working as expected! As mentioned, we can use the `PineconeIndex` for persistance and high scale use-cases, for example where we might have hundreds of thousands of utterances, or even millions.

---